In [7]:
from __future__ import print_function, division, absolute_import # python 2 compatibility
import tensorflow as tf
import apache_beam as beam
import shutil
import os
print(tf.__version__)

1.9.0


Required if using Cloud ML Engine to run the model, otherwise not required

In [0]:
REGION = 'asia-east1'
BUCKET = '{BUCKET}'
PROJECT = '{PROJECT}'
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.9'

In [0]:
%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

---